<a href="https://colab.research.google.com/github/robonesky/oreilly/blob/main/RM%20Lab%20Notes%20Advanced%20ML-%20part%20III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=f8116fbee33a47dcd4b538511419a62dd36d7dbc2ea194ac9100e4b508d94c0b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
%pip install boruta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 5.2 MB/s eta 0:00:00


In [5]:
%pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.8 MB/s eta 0:00:00


In [6]:
# Machine Learning Workshop - I
# Kannan Singaravelu

# basic imports
import os, random
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
from pathlib import Path

# import boruta
from boruta import BorutaPy

# warnings
import warnings
warnings.filterwarnings('ignore')

# plotting & outputs
from pprint import pprint
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# functions from helper
# from src.helper import *

# import custom transformer
# from helper import DayTransformer, TimeTransformer

# sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import auc, roc_curve #, plot_roc_curve # plot_confusion_matrix

# import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier

# tensorflow
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Accuracy, AUC, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.layers import LSTM, BatchNormalization

# kerastuner
import keras_tuner as kt
from kerastuner import HyperParameter, HyperParameters
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband

from sklearn.base import BaseEstimator, TransformerMixin

# Machine Learning Workshop - I by Kannan Singaravelu
# November 2021


In [7]:
# Machine Learning Workshop - I
# Kannan Singaravelu

import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin


# define seed
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


# create custom day transformer
class DayTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.data = pd.DataFrame(
            {
        'WeekDay': ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
            }
        )
        self.daysnum = np.array(self.data.index+1)

        print(f'self.daysnum is: {self.daysnum}')

        return self


    def transform(self, X): # X is a dataframe
        Xt = X.copy()
        pi = np.pi
        num = Xt.index.weekday+1

        Xt['dsin'] = np.sin(2 * pi * num / np.max(self.daysnum))
        Xt['dcos'] = np.cos(2 * pi * num / np.max(self.daysnum))
        Xt = Xt.drop(['days'], axis=1)

        return Xt


# create custom time transformer
class TimeTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.data = pd.DataFrame(
            {
        'DayParts': ["afternoon","morning","noon"]
            }
        )
        self.timenum = np.array(self.data.index+1)
        return self


    def transform(self, X):
        Xt = X.copy()
        pi = np.pi
        num = Xt.hours.apply(lambda x: 1 if x=='afternoon' else (2 if x=='morning' else 3))
        Xt['tsin'] = np.sin(2 * pi * num / np.max(self.timenum))
        Xt['tcos'] = np.cos(2 * pi * num / np.max(self.timenum))
        Xt = Xt.drop(['hours'], axis=1)

        return Xt


# create function to read locally stored file
def getdata(filename):
    df = pd.read_csv('./'+filename+'.csv')
    df.datetime = pd.to_datetime(df.datetime)
    df = (
        df.set_index('datetime', drop=True)
        .drop('symbol', axis=1)
    )

    # add days
    df['days'] = df.index.day_name()

    # add dayparts
    df['hours'] = df.index.hour
    df['hours'] = df['hours'].apply(daypart)

    return df


# create function to group trade hours
def daypart(hour):
    if hour in [9,10,11]:
        return "morning"
    elif hour in [12,13]:
        return "noon"
    elif hour in [14,15,16,17,18,19]:
        return "afternoon"


# class weight function
def cwts(dfs):
    c0, c1 = np.bincount(dfs)
    w0=(1/c0)*(len(dfs))/2
    w1=(1/c1)*(len(dfs))/2
    return {0: w0, 1: w1}

# Machine Learning Workshop - I by Kannan Singaravelu
# November 2021

In [8]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
results_path = Path('results', 'lstm_time_series')
if not results_path.exists():
    results_path.mkdir(parents=True)

df = getdata('NFUT1H')

type(df)

pandas.core.frame.DataFrame

In [12]:
type(df['close'])

pandas.core.series.Series

In [16]:
type(df[['close']])

pandas.core.frame.DataFrame

In [19]:
df = df [['close']]
df['ret'] = np.log(df).diff().fillna(0)
df['dir'] = np.where(
    df.ret.shift(-1)>0,1,0
    )

df.tail(10)

,close,ret,dir
datetime,,,
2021-07-13 13:00:00,15830.00,0.000752,0
2021-07-13 14:00:00,15820.25,-0.000616,1
2021-07-13 15:00:00,15833.95,0.000866,0
2021-07-14 09:00:00,15784.00,-0.003160,1
2021-07-14 10:00:00,15839.40,0.003504,1
2021-07-14 11:00:00,15848.40,0.000568,1
2021-07-14 12:00:00,15874.20,0.001627,0
2021-07-14 13:00:00,15874.00,-0.000013,1
2021-07-14 14:00:00,15878.35,0.000274,0


In [20]:
cols = []

for lag in[1, 2, 3, 4, 5, 6, 7]:
  col = f'ret_{lag}'
  df[col] = df.ret.shift(lag)
  cols.append(col)

df.dropna(inplace = True)

# these seem to capture if the return was + or -, but in a binary way
cols_bin = []
for col in cols:
  col_bin = f'{col}_bin'
  df[col_bin] = np.digitize(df[col], bins=[0])
  cols_bin.append(col_bin)


df.tail(10)

,close,ret,dir,ret_1,ret_2,ret_3,ret_4,ret_5,ret_6,ret_7,ret_1_bin,ret_2_bin,ret_3_bin,ret_4_bin,ret_5_bin,ret_6_bin,ret_7_bin
datetime,,,,,,,,,,,,,,,,,
2021-07-13 13:00:00,15830.00,0.000752,0,0.000177,0.002865,-0.000910,0.004734,0.000786,-0.001241,-0.004190,1,1,0,1,1,0,0
2021-07-13 14:00:00,15820.25,-0.000616,1,0.000752,0.000177,0.002865,-0.000910,0.004734,0.000786,-0.001241,1,1,1,0,1,1,0
2021-07-13 15:00:00,15833.95,0.000866,0,-0.000616,0.000752,0.000177,0.002865,-0.000910,0.004734,0.000786,0,1,1,1,0,1,1
2021-07-14 09:00:00,15784.00,-0.003160,1,0.000866,-0.000616,0.000752,0.000177,0.002865,-0.000910,0.004734,1,0,1,1,1,0,1
2021-07-14 10:00:00,15839.40,0.003504,1,-0.003160,0.000866,-0.000616,0.000752,0.000177,0.002865,-0.000910,0,1,0,1,1,1,0
2021-07-14 11:00:00,15848.40,0.000568,1,0.003504,-0.003160,0.000866,-0.000616,0.000752,0.000177,0.002865,1,0,1,0,1,1,1
2021-07-14 12:00:00,15874.20,0.001627,0,0.000568,0.003504,-0.003160,0.000866,-0.000616,0.000752,0.000177,1,1,0,1,0,1,1
2021-07-14 13:00:00,15874.00,-0.000013,1,0.001627,0.000568,0.003504,-0.003160,0.000866,-0.000616,0.000752,1,1,1,0,1,0,1
2021-07-14 14:00:00,15878.35,0.000274,0,-0.000013,0.001627,0.000568,0.003504,-0.003160,0.000866,-0.000616,0,1,1,1,0,1,0


In [23]:
X = df[cols_bin] # just taking teh binary ones it seems
X

,ret_1_bin,ret_2_bin,ret_3_bin,ret_4_bin,ret_5_bin,ret_6_bin,ret_7_bin
datetime,,,,,,,
2018-08-17 14:00:00,1,0,0,1,1,1,1
2018-08-17 15:00:00,1,1,0,0,1,1,1
2018-08-20 09:00:00,0,1,1,0,0,1,1
2018-08-20 10:00:00,1,0,1,1,0,0,1
2018-08-20 11:00:00,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...
2021-07-14 11:00:00,1,0,1,0,1,1,1
2021-07-14 12:00:00,1,1,0,1,0,1,1
2021-07-14 13:00:00,1,1,1,0,1,0,1


In [24]:
y = df['dir'] # the target variable
y

datetime
2018-08-17 14:00:00    0
2018-08-17 15:00:00    1
2018-08-20 09:00:00    0
2018-08-20 10:00:00    1
2018-08-20 11:00:00    1
                      ..
2021-07-14 11:00:00    1
2021-07-14 12:00:00    0
2021-07-14 13:00:00    1
2021-07-14 14:00:00    0
2021-07-14 15:00:00    0
Name: dir, Length: 4993, dtype: int64

In [25]:
# look for class imbalance
c = np.bincount(y)
c

array([2359, 2634])

In [26]:
# not drastically bad, but still imbalanced
class_weight = cwts(y)
class_weight

{0: 1.0582874099194575, 1: 0.9477980258162491}

In [27]:
class_weight[0] * c[0], class_weight[1] * c[1]

(2496.5, 2496.5)

In [28]:
# Splitting the datasets into training and testing data.
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, shuffle=False)
# train, test = train_test_split(X, train_size=0.8, test_size=0.2, shuffle=False)

# Output the train and test data size
print(f"Train and Test Size {Xtrain.shape}, {ytrain.shape}, {Xtest.shape}, {ytest.shape}")
# print(f"Train and Test Size {train.shape}, {test.shape}")}

Train and Test Size (3994, 7), (3994,), (999, 7), (999,)


In [30]:
# feature scaling

scaler = MinMaxScaler()
scaledtrain = scaler.fit_transform(Xtrain)
scaledtest = scaler.transform(Xtest)

In [34]:
seqlen = 21
numfeat = X.shape[1]#

# Note how we're using TimeSeriesGenerator now
g = TimeseriesGenerator(scaledtrain, ytrain, length = seqlen)
g_ = TimeseriesGenerator(scaledtest, ytest, length = seqlen)

len(g), len(g_)

(32, 8)

In [35]:
g[0][0]

array([[[1., 0., 0., ..., 1., 1., 1.],
        [1., 1., 0., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 1., 1.],
        ...,
        [0., 0., 1., ..., 0., 1., 0.],
        [1., 0., 0., ..., 1., 0., 1.],
        [0., 1., 0., ..., 1., 1., 0.]],

       [[1., 1., 0., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 1., 1.],
        [1., 0., 1., ..., 0., 0., 1.],
        ...,
        [1., 0., 0., ..., 1., 0., 1.],
        [0., 1., 0., ..., 1., 1., 0.],
        [1., 0., 1., ..., 0., 1., 1.]],

       [[0., 1., 1., ..., 0., 1., 1.],
        [1., 0., 1., ..., 0., 0., 1.],
        [0., 1., 0., ..., 1., 0., 0.],
        ...,
        [0., 1., 0., ..., 1., 1., 0.],
        [1., 0., 1., ..., 0., 1., 1.],
        [1., 1., 0., ..., 0., 0., 1.]],

       ...,

       [[1., 0., 1., ..., 0., 1., 1.],
        [1., 1., 0., ..., 1., 0., 1.],
        [1., 1., 1., ..., 1., 1., 0.],
        ...,
        [1., 0., 0., ..., 0., 1., 0.],
        [1., 1., 0., ..., 0., 0., 1.],
        [1., 1., 1., ..., 0., 0.